In [2]:
import vertica_python 
import datetime as dt
#import itertools as it
import numpy as np
import pandas as pd
import os

#### Будем создавать исходные переменные для модели прямо в Вертике для того, чтобы потом оттуда все выкачать и заджойнить в финальной моделе

In [3]:
conn_vert = vertica_python.connect(host = host, port=5433, user=user, password=pass, database=base)
cur_vert = conn_vert.cursor()

In [1]:
# 2queries, drop table churn.week_start, create table churn.week_start with new weeks
query_create_week_start = """ 
-- recreate churn.week_start with new dates
drop table if exists churn.week_start
;
create table churn.week_start as
select distinct(date(date_trunc('week', event_date))) as week_start from mobile_sessions
"""

In [17]:
%%time
cur_vert.execute(query_create_week_start)

CPU times: user 1.92 ms, sys: 1.46 ms, total: 3.37 ms
Wall time: 5.04 s


In [14]:
query_create_churn_mobile_clients = """
drop table if exists churn.mobile_clients
;
create table churn.mobile_clients as
select distinct(customer_id) from mobile_sessions
;"""

In [15]:
%%time
cur_vert.execute(query_create_churn_mobile_clients)

CPU times: user 2.61 ms, sys: 2.14 ms, total: 4.75 ms
Wall time: 33.3 s


In [8]:
# 3 queries, delete last week, update for old users new week, add new users with all weeks
query_update_clients_week_start = """
-- drop last week due to probably incomplete
delete from churn.clients_week_start where week_start = (select max(week_start) from churn.clients_week_start)
;
-- insert last weeks for previous users
-- IT SHOULD BE FIRST before inserting all weeks for newer users
insert into churn.clients_week_start select * from ( 
    select a.customer_id, b.week_start from (
        (select distinct(customer_id) from churn.clients_week_start) 
    ) a 
    cross join (select * from churn.week_start where week_start > (select max(week_start) from churn.clients_week_start)) b
) c
;
-- insert all weeks for users who come in last periods
insert into churn.clients_week_start select * from ( 
    select a.customer_id, b.week_start from (
        (select distinct(customer_id) from mobile_sessions 
            where event_date > (select max(week_start) from churn.clients_week_start)
            and customer_id not in (select distinct(customer_id) from churn.clients_week_start)
        ) 
    ) a 
    cross join churn.week_start b
) c
"""

In [9]:
%%time
cur_vert.execute(query_update_clients_week_start)

In [6]:
# another variant to create clients_week_start from scratch
query_create_clients_week_start = """
drop table if exists churn.clients_week_start
;
create table churn.clients_week_start as
select a.customer_id as client_pin, b.week_start from (select distinct(customer_id) from mobile_sessions) a 
cross join churn.week_start b 
"""

In [7]:
# %%time
# cur_vert.execute(query_create_clients_week_start)

CPU times: user 5.52 ms, sys: 7.03 ms, total: 12.6 ms
Wall time: 3min 35s


In [11]:
# delete records for last week and add everything till now
query_update_churn_feature_target="""
delete from churn.churn_feature_target where week_start = (select max(week_start) from churn.churn_feature_target)
;
insert into churn.churn_feature_target select * from (
    select a.*, case when b.exist = 1 then 1 else 0 end as target 
    from churn.clients_week_start a
    left join
    (select 1 as exist, customer_id as client_pin, timestampadd('week', -1, (date(date_trunc('week', event_date)))) as week_start
        from mobile_sessions 
        where operation like 'Successful%login' and event_date > (select max(week_start) from churn.churn_feature_target)
        group by customer_id, week_start
        order by week_start) b
    on a.client_pin = b.client_pin and a.week_start = b.week_start 
 ) c
;
"""

In [8]:
## create whole table from scratch
query_create_churn_feature_target = """
drop table if exists churn.churn_feature_target
;
create table churn.churn_feature_target as 
select a.*, case when b.exist = 1 then 1 else 0 end as target 
from churn.clients_week_start a
left join
(select 1 as exist, customer_id as client_pin, timestampadd('week', -1, (date(date_trunc('week', event_date)))) as week_start
    from mobile_sessions 
    where operation like 'Successful%login'
    group by customer_id, week_start
    order by week_start) b
on a.client_pin = b.client_pin and a.week_start = b.week_start
"""

In [9]:
# %%time
# cur_vert.execute(query_create_churn_feature_target)

CPU times: user 11.2 ms, sys: 11.9 ms, total: 23.1 ms
Wall time: 9min 16s


In [10]:
## drop and create churn_feature_num_cards from scratch
query_create_churn_feature_num_cards = """
drop table if exists churn.churn_feature_num_cards
;
create table churn.churn_feature_num_cards as
select 
    week_start, 
    client_pin, 
    -- count debit and credit cards
    sum(case when a.week_start > b.start_date and a.week_start < b.expire_date and cardtype_ccode='D' then 1 else 0 end) as count_debit,
    sum(case when a.week_start > b.start_date and a.week_start < b.expire_date and cardtype_ccode='C' then 1 else 0 end) as count_credit
        from 
            (select week_start, client_pin from churn.clients_week_start) a
            left join 
                -- select cards which was issued and not deleted
                (select client_owner_pin, start_date, expire_date, cardtype_ccode from oracle.card_sdim
                where 
                    expire_date > '2015-01-01' 
                    and issuedcard_flag = 'Y'
                    and deleted_flag = 'N'
                ) b
            on a.client_pin = b.client_owner_pin
group by week_start, client_pin
"""

In [11]:
%%time
cur_vert.execute(query_create_churn_feature_num_cards)

CPU times: user 8.69 ms, sys: 9.85 ms, total: 18.5 ms
Wall time: 7min 35s


In [16]:
query_create_churn_feature_info_clients = """create table churn.churn_feature_info_clients as
select  
    a.client_pin,
    a.week_start,
    b.gender_uk,
    b.country_uk,
    (a.week_start - date(b.birth_date))/365.2425 as age,
    a.week_start - date(b.start_date) as client_life_span
from churn.clients_week_start a
left join oracle.client_sdim b
on a.client_pin = b.client_pin
where a.week_start - date(b.start_date) >= 0"""

In [17]:
%%time
cur_vert.execute(query_create_churn_feature_info_clients)

CPU times: user 9.57 ms, sys: 10.5 ms, total: 20 ms
Wall time: 8min 2s
